# Infer-20-Decision-Sequential : MDPs, Bandits et POMDPs

**Serie** : Programmation Probabiliste avec Infer.NET (20/20)  
**Duree estimee** : 60 minutes  
**Prerequis** : Notebooks 14-19 (Decision Theory)

---

## Objectifs

- Comprendre les **Processus de Decision Markoviens** (MDPs)
- Maitriser l'**iteration de valeur** et l'**iteration de politique**
- Decouvrir les alternatives : **LP, Expectimax, RTDP**
- Appliquer le **reward shaping** avec le theoreme de preservation de politique
- Introduire les **bandits multi-bras** et l'**indice de Gittins**
- Presenter les **POMDPs** et les belief states

---

## Navigation

| Precedent | Suivant |
|-----------|--------|
| [Infer-19-Decision-Expert-Systems](Infer-19-Decision-Expert-Systems.ipynb) | Serie RL (`MyIA.AI.Notebooks/RL/`) |

---

**Note** : Ce notebook est une introduction conceptuelle aux decisions sequentielles. Les implementations avancees avec Deep RL sont dans la serie `RL/`.

## 1. Decisions Sequentielles vs One-Shot

### Difference fondamentale

| Type | Caracteristique | Exemple |
|------|-----------------|--------|
| **One-shot** | Une seule decision | Accepter une offre d'emploi |
| **Sequentielle** | Serie de decisions interdependantes | Jouer aux echecs |

### Horizon

- **Fini** : Nombre fixe d'etapes (T etapes)
- **Infini** : Le processus continue indefiniment

### Facteur d'actualisation γ (gamma)

Pour les horizons infinis, on utilise un facteur γ ∈ [0,1) pour :

- Garantir la convergence des sommes infinies
- Modeliser la preference pour les recompenses immediates
- γ = 0.99 : vision long terme
- γ = 0.5 : vision court terme

In [9]:
// Installation Infer.NET
#r "nuget: Microsoft.ML.Probabilistic"
#r "nuget: Microsoft.ML.Probabilistic.Compiler"

using Microsoft.ML.Probabilistic;
using Microsoft.ML.Probabilistic.Distributions;
using Microsoft.ML.Probabilistic.Models;
using Microsoft.ML.Probabilistic.Algorithms;

Console.WriteLine("Infer.NET charge !");

Installed Packages Microsoft.ML.Probabilistic, 0.4.2504.701 Microsoft.ML.Probabilistic.Compiler, 0.4.2504.701

Infer.NET charge !


## 2. Processus de Decision Markoviens (MDP)

### Definition formelle

Un MDP est un tuple (S, A, P, R, γ) :

| Element | Notation | Description |
|---------|----------|-------------|
| **Etats** | S | Ensemble des etats possibles |
| **Actions** | A | Ensemble des actions |
| **Transitions** | P(s'\|s,a) | Probabilite de transition |
| **Recompenses** | R(s,a) ou R(s,a,s') | Recompense immediate |
| **Discount** | γ | Facteur d'actualisation |

### Politique et Fonction de Valeur

- **Politique** π : S → A (ou distribution sur A)
- **Fonction de valeur** V^π(s) : Utilite esperee depuis s en suivant π
- **Fonction action-valeur** Q^π(s, a) : Utilite esperee en faisant a puis suivant π

In [10]:
// Definition d'un MDP simple : Navigation dans une grille

public class GridMDP
{
    public int Width { get; }
    public int Height { get; }
    public double Gamma { get; }
    public Dictionary<(int, int), double> Rewards { get; }
    public HashSet<(int, int)> TerminalStates { get; }
    public HashSet<(int, int)> Walls { get; }
    
    public string[] Actions { get; } = { "N", "S", "E", "W" };
    
    // Directions
    private Dictionary<string, (int dx, int dy)> _directions = new()
    {
        { "N", (0, 1) }, { "S", (0, -1) }, { "E", (1, 0) }, { "W", (-1, 0) }
    };
    
    public GridMDP(int width, int height, double gamma = 0.9)
    {
        Width = width;
        Height = height;
        Gamma = gamma;
        Rewards = new Dictionary<(int, int), double>();
        TerminalStates = new HashSet<(int, int)>();
        Walls = new HashSet<(int, int)>();
    }
    
    public List<(int, int)> GetStates()
    {
        var states = new List<(int, int)>();
        for (int x = 0; x < Width; x++)
            for (int y = 0; y < Height; y++)
                if (!Walls.Contains((x, y)))
                    states.Add((x, y));
        return states;
    }
    
    public double GetReward((int, int) state) => 
        Rewards.TryGetValue(state, out var r) ? r : -0.04; // Cout de mouvement par defaut
    
    public List<((int, int) nextState, double prob)> GetTransitions((int, int) state, string action)
    {
        if (TerminalStates.Contains(state))
            return new List<((int, int), double)> { (state, 1.0) };
        
        var transitions = new List<((int, int), double)>();
        var (dx, dy) = _directions[action];
        
        // 80% de chance d'aller dans la direction voulue
        transitions.Add((NextState(state, action), 0.8));
        
        // 10% de chance d'aller perpendiculairement (droite ou gauche)
        var perpActions = action == "N" || action == "S" 
            ? new[] { "E", "W" } 
            : new[] { "N", "S" };
        
        foreach (var perpAction in perpActions)
            transitions.Add((NextState(state, perpAction), 0.1));
        
        return transitions;
    }
    
    private (int, int) NextState((int, int) state, string action)
    {
        var (x, y) = state;
        var (dx, dy) = _directions[action];
        int nx = x + dx, ny = y + dy;
        
        // Collision avec mur ou bord = rester sur place
        if (nx < 0 || nx >= Width || ny < 0 || ny >= Height || Walls.Contains((nx, ny)))
            return state;
        return (nx, ny);
    }
}

// Creer le MDP classique de Russell & Norvig
var mdp = new GridMDP(4, 3, gamma: 0.9);
mdp.Rewards[(3, 2)] = 1.0;   // But positif
mdp.Rewards[(3, 1)] = -1.0;  // Piege
mdp.TerminalStates.Add((3, 2));
mdp.TerminalStates.Add((3, 1));
mdp.Walls.Add((1, 1));       // Mur

Console.WriteLine("MDP Grille 4x3 cree :");
Console.WriteLine("  But (+1) en (3,2)");
Console.WriteLine("  Piege (-1) en (3,1)");
Console.WriteLine("  Mur en (1,1)");
Console.WriteLine($"  Gamma = {mdp.Gamma}");

MDP Grille 4x3 cree :
  But (+1) en (3,2)
  Piege (-1) en (3,1)
  Mur en (1,1)
  Gamma = 0,9


## 3. Equation de Bellman

### Intuition

L'equation de Bellman exprime un principe fondamental de **consistance temporelle** :

> La valeur d'un etat = recompense immediate + valeur actualisee des etats futurs

C'est une relation **recursive** : la valeur de chaque etat depend de la valeur des etats atteignables. Cette structure permet de resoudre le probleme "a l'envers" (backward induction) ou iterativement.

### Equation de Bellman pour V*

La fonction de valeur optimale satisfait :

$$V^*(s) = \max_a \left[ R(s,a) + \gamma \sum_{s'} P(s'|s,a) V^*(s') \right]$$

**Interpretation terme par terme** :
- $\max_a$ : On choisit la meilleure action
- $R(s,a)$ : Recompense immediate
- $\gamma$ : Facteur d'actualisation (valeur du futur)
- $\sum_{s'} P(s'|s,a) V^*(s')$ : Esperance de la valeur future

### Equation de Bellman pour Q*

$$Q^*(s, a) = R(s,a) + \gamma \sum_{s'} P(s'|s,a) \max_{a'} Q^*(s', a')$$

La fonction Q donne la valeur de **faire a puis agir optimalement**.

### Politique optimale

$$\pi^*(s) = \arg\max_a Q^*(s, a)$$

La politique optimale est **deterministe** : dans chaque etat, une action domine.

### Pourquoi Bellman est-il si important ?

L'equation de Bellman est la **cle de voute** de la programmation dynamique et du reinforcement learning car :

1. Elle decompose un probleme global en sous-problemes locaux
2. Elle fournit une condition d'optimalite verifiable
3. Elle inspire les algorithmes (VI, PI, Q-learning, etc.)

## 4. Iteration de Valeur

### Algorithme

1. Initialiser V(s) = 0 pour tout s
2. Repeter jusqu'a convergence :
   - Pour chaque etat s :
     - V(s) ← max_a [R(s,a) + γ Σ_s' P(s'|s,a) V(s')]
3. Extraire la politique : π(s) = argmax_a Q(s,a)

### Complexite

- O(|S|² |A|) par iteration
- Converge en O(log(1/ε) / (1-γ)) iterations

In [11]:
// Iteration de Valeur

public class ValueIteration
{
    public static (Dictionary<(int,int), double> V, Dictionary<(int,int), string> policy) 
        Solve(GridMDP mdp, double epsilon = 0.001, int maxIter = 100)
    {
        var states = mdp.GetStates();
        var V = states.ToDictionary(s => s, s => 0.0);
        
        for (int iter = 0; iter < maxIter; iter++)
        {
            double delta = 0;
            var newV = new Dictionary<(int, int), double>(V);
            
            foreach (var s in states)
            {
                if (mdp.TerminalStates.Contains(s))
                {
                    newV[s] = mdp.GetReward(s);
                    continue;
                }
                
                double maxQ = double.NegativeInfinity;
                foreach (var a in mdp.Actions)
                {
                    double q = mdp.GetReward(s);
                    foreach (var (nextState, prob) in mdp.GetTransitions(s, a))
                        q += mdp.Gamma * prob * V[nextState];
                    maxQ = Math.Max(maxQ, q);
                }
                
                newV[s] = maxQ;
                delta = Math.Max(delta, Math.Abs(V[s] - newV[s]));
            }
            
            V = newV;
            
            if (delta < epsilon)
            {
                Console.WriteLine($"Convergence apres {iter + 1} iterations (delta = {delta:E2})");
                break;
            }
        }
        
        // Extraire la politique
        var policy = new Dictionary<(int, int), string>();
        foreach (var s in states)
        {
            if (mdp.TerminalStates.Contains(s))
            {
                policy[s] = "T"; // Terminal
                continue;
            }
            
            string bestAction = null;
            double maxQ = double.NegativeInfinity;
            
            foreach (var a in mdp.Actions)
            {
                double q = mdp.GetReward(s);
                foreach (var (nextState, prob) in mdp.GetTransitions(s, a))
                    q += mdp.Gamma * prob * V[nextState];
                
                if (q > maxQ)
                {
                    maxQ = q;
                    bestAction = a;
                }
            }
            policy[s] = bestAction;
        }
        
        return (V, policy);
    }
}

var (V, policy) = ValueIteration.Solve(mdp);

// Afficher la grille
Console.WriteLine("\nFonction de Valeur V* :");
for (int y = mdp.Height - 1; y >= 0; y--)
{
    for (int x = 0; x < mdp.Width; x++)
    {
        if (mdp.Walls.Contains((x, y)))
            Console.Write("  ####  ");
        else
            Console.Write($" {V[(x, y)],6:F3} ");
    }
    Console.WriteLine();
}

Console.WriteLine("\nPolitique optimale π* :");
var arrows = new Dictionary<string, string> { {"N","↑"}, {"S","↓"}, {"E","→"}, {"W","←"}, {"T","●"} };
for (int y = mdp.Height - 1; y >= 0; y--)
{
    for (int x = 0; x < mdp.Width; x++)
    {
        if (mdp.Walls.Contains((x, y)))
            Console.Write(" # ");
        else
            Console.Write($" {arrows[policy[(x, y)]]} ");
    }
    Console.WriteLine();
}

Convergence apres 14 iterations (delta = 6,01E-004)

Fonction de Valeur V* :
  0,509   0,650   0,795   1,000 
  0,398   ####    0,486  -1,000 
  0,296   0,254   0,345   0,130 

Politique optimale π* :
 →  →  →  ● 
 ↑  #  ↑  ● 
 ↑  →  ↑  ← 


## 5. Iteration de Politique

### Algorithme

1. Initialiser π arbitrairement
2. Repeter jusqu'a stabilite :
   - **Evaluation** : Calculer V^π (resoudre systeme lineaire)
   - **Amelioration** : π(s) ← argmax_a Q^π(s,a)

### Compromis Value Iteration vs Policy Iteration

| Critere | Value Iteration | Policy Iteration |
|---------|-----------------|------------------|
| **Iterations** | Beaucoup (log(1/ε)/(1-γ)) | Peu (souvent 5-10) |
| **Cout/iteration** | O(\|S\|² \|A\|) leger | O(\|S\|³) evaluation exacte |
| **Convergence** | Asymptotique (ε-optimal) | Exacte en nombre fini |
| **Memoire** | V(s) seulement | V(s) + π(s) |

### Quand utiliser chaque methode ?

**Preferer Value Iteration si** :
- Grand espace d'etats (\|S\| > 10,000)
- Precision approximative suffisante
- γ proche de 1 (convergence lente de PI)
- Implementation simple souhaitee

**Preferer Policy Iteration si** :
- Petit/moyen espace d'etats
- Solution exacte requise
- Politique stable rapidement
- Evaluation peut etre acceleree (ex: methodes matricielles)

### Variantes hybrides

- **Modified Policy Iteration** : Evaluation partielle (k iterations de Bellman au lieu de resolution exacte)
- **Asynchronous VI** : Mise a jour d'un sous-ensemble d'etats a chaque iteration
- **Prioritized Sweeping** : Mettre a jour les etats les plus "surprenants" en premier

In [12]:
// Iteration de Politique

public class PolicyIteration
{
    public static (Dictionary<(int,int), double> V, Dictionary<(int,int), string> policy) 
        Solve(GridMDP mdp, int maxIter = 20)
    {
        var states = mdp.GetStates();
        
        // Initialiser politique aleatoire
        var policy = states.ToDictionary(s => s, s => 
            mdp.TerminalStates.Contains(s) ? "T" : "N");
        
        var V = states.ToDictionary(s => s, s => 0.0);
        
        for (int iter = 0; iter < maxIter; iter++)
        {
            // 1. Evaluation de politique (iterative simplifiee)
            for (int evalIter = 0; evalIter < 50; evalIter++)
            {
                var newV = new Dictionary<(int, int), double>(V);
                foreach (var s in states)
                {
                    if (mdp.TerminalStates.Contains(s))
                    {
                        newV[s] = mdp.GetReward(s);
                        continue;
                    }
                    
                    string a = policy[s];
                    double v = mdp.GetReward(s);
                    foreach (var (nextState, prob) in mdp.GetTransitions(s, a))
                        v += mdp.Gamma * prob * V[nextState];
                    newV[s] = v;
                }
                V = newV;
            }
            
            // 2. Amelioration de politique
            bool stable = true;
            foreach (var s in states)
            {
                if (mdp.TerminalStates.Contains(s)) continue;
                
                string oldAction = policy[s];
                string bestAction = null;
                double maxQ = double.NegativeInfinity;
                
                foreach (var a in mdp.Actions)
                {
                    double q = mdp.GetReward(s);
                    foreach (var (nextState, prob) in mdp.GetTransitions(s, a))
                        q += mdp.Gamma * prob * V[nextState];
                    
                    if (q > maxQ)
                    {
                        maxQ = q;
                        bestAction = a;
                    }
                }
                
                policy[s] = bestAction;
                if (bestAction != oldAction) stable = false;
            }
            
            Console.WriteLine($"Iteration {iter + 1} : stable = {stable}");
            if (stable) break;
        }
        
        return (V, policy);
    }
}

Console.WriteLine("=== Iteration de Politique ===\n");
var (V_pi, policy_pi) = PolicyIteration.Solve(mdp);

// Verifier que les resultats sont identiques
Console.WriteLine("\nComparaison avec Value Iteration :");
bool same = true;
foreach (var s in mdp.GetStates())
{
    if (policy[s] != policy_pi[s])
    {
        same = false;
        Console.WriteLine($"  Difference en {s}: VI={policy[s]}, PI={policy_pi[s]}");
    }
}
if (same) Console.WriteLine("  Politiques identiques !");

=== Iteration de Politique ===

Iteration 1 : stable = False
Iteration 2 : stable = False
Iteration 3 : stable = True

Comparaison avec Value Iteration :
  Politiques identiques !


## 6. Alternatives : LP, Expectimax, RTDP

### Programmation Lineaire (LP)

Le MDP peut etre formule comme un programme lineaire :

- **Variables** : V(s) pour chaque etat
- **Objectif** : min Σ_s V(s)
- **Contraintes** : V(s) ≥ R(s,a) + γ Σ_s' P(s'|s,a) V(s') pour tout a

### Expectimax

Pour les MDPs a horizon fini, on peut utiliser un arbre de recherche :

```
       (s0)
      / | \
   a1  a2  a3     <- Noeuds max (choix agent)
   / \ 
 s1  s2           <- Noeuds chance (transition)
```

### RTDP (Real-Time Dynamic Programming)

- Mise a jour le long de trajectoires simulees
- Focus sur les etats atteignables
- Algorithme **anytime** : ameliore avec plus de temps

In [13]:
// RTDP simplifie

public class RTDP
{
    public static Dictionary<(int,int), double> Solve(
        GridMDP mdp, (int, int) startState, int nTrials = 100)
    {
        var states = mdp.GetStates();
        var V = states.ToDictionary(s => s, s => 0.0);
        var rng = new Random(42);
        
        for (int trial = 0; trial < nTrials; trial++)
        {
            var s = startState;
            int steps = 0;
            
            while (!mdp.TerminalStates.Contains(s) && steps < 100)
            {
                // Mise a jour de Bellman
                double maxQ = double.NegativeInfinity;
                string bestAction = null;
                
                foreach (var a in mdp.Actions)
                {
                    double q = mdp.GetReward(s);
                    foreach (var (nextState, prob) in mdp.GetTransitions(s, a))
                        q += mdp.Gamma * prob * V[nextState];
                    
                    if (q > maxQ)
                    {
                        maxQ = q;
                        bestAction = a;
                    }
                }
                
                V[s] = maxQ;
                
                // Simuler transition
                var transitions = mdp.GetTransitions(s, bestAction);
                double r = rng.NextDouble();
                double cumProb = 0;
                foreach (var (nextState, prob) in transitions)
                {
                    cumProb += prob;
                    if (r <= cumProb)
                    {
                        s = nextState;
                        break;
                    }
                }
                
                steps++;
            }
            
            // Update terminal state
            if (mdp.TerminalStates.Contains(s))
                V[s] = mdp.GetReward(s);
        }
        
        return V;
    }
}

Console.WriteLine("=== RTDP (100 trials depuis (0,0)) ===\n");
var V_rtdp = RTDP.Solve(mdp, (0, 0), nTrials: 100);

// Comparer avec Value Iteration
Console.WriteLine("Comparaison V_RTDP vs V_VI :");
Console.WriteLine("  Etat   |  V_RTDP  |   V_VI   |   Diff");
Console.WriteLine("---------|----------|----------|--------");
foreach (var s in new[] { (0,0), (0,2), (2,2), (3,2) })
{
    double diff = Math.Abs(V_rtdp[s] - V[s]);
    Console.WriteLine($" ({s.Item1},{s.Item2})   | {V_rtdp[s],8:F4} | {V[s],8:F4} | {diff,6:F4}");
}

=== RTDP (100 trials depuis (0,0)) ===

Comparaison V_RTDP vs V_VI :
  Etat   |  V_RTDP  |   V_VI   |   Diff
---------|----------|----------|--------
 (0,0)   |   0,1666 |   0,2960 | 0,1294
 (0,2)   |  -0,1123 |   0,5094 | 0,6217
 (2,2)   |   0,7954 |   0,7954 | 0,0000
 (3,2)   |   1,0000 |   1,0000 | 0,0000


## 7. Reward Shaping

### Le probleme des recompenses sparses

Quand les recompenses sont rares (ex: +1 seulement au but), l'apprentissage est tres lent.

### Solution : Ajouter une recompense de faconnage

$$R'(s, a, s') = R(s, a, s') + F(s, a, s')$$

### Theoreme de preservation de politique (Ng et al., 1999)

> Si F a la forme d'une **fonction de potentiel** :
> 
> $$F(s, a, s') = \gamma \Phi(s') - \Phi(s)$$
> 
> Alors **la politique optimale est preservee** !

### Intuition

- Φ(s) represente "a quel point s est proche du but"
- F recompense les transitions vers des etats meilleurs
- La forme specifique garantit que les raccourcis ne sont pas crees

In [14]:
// Demonstration du Reward Shaping

public class ShapedGridMDP : GridMDP
{
    private Func<(int, int), double> _potential;
    private double _gamma;
    
    public ShapedGridMDP(int width, int height, double gamma, Func<(int, int), double> potential) 
        : base(width, height, gamma)
    {
        _potential = potential;
        _gamma = gamma;
    }
    
    public double GetShapedReward((int, int) s, (int, int) sPrime)
    {
        double R = GetReward(s);
        double F = _gamma * _potential(sPrime) - _potential(s);
        return R + F;
    }
}

// Fonction de potentiel : distance negative au but
(int, int) goal = (3, 2);
Func<(int, int), double> Phi = s => -Math.Sqrt(Math.Pow(s.Item1 - goal.Item1, 2) + Math.Pow(s.Item2 - goal.Item2, 2));

Console.WriteLine("=== Reward Shaping avec Fonction de Potentiel ===\n");
Console.WriteLine($"But en {goal}");
Console.WriteLine("Phi(s) = -distance(s, but)\n");

Console.WriteLine("Fonction de potentiel Phi(s) :");
for (int y = 2; y >= 0; y--)
{
    for (int x = 0; x < 4; x++)
    {
        if (mdp.Walls.Contains((x, y)))
            Console.Write("  ####  ");
        else
            Console.Write($" {Phi((x, y)),6:F2} ");
    }
    Console.WriteLine();
}

Console.WriteLine("\nExemples de shaping reward F(s, a, s') = gamma*Phi(s') - Phi(s) :");
Console.WriteLine($"  F((0,0) -> (1,0)) = {0.9 * Phi((1, 0)) - Phi((0, 0)):F3} (vers le but)");
Console.WriteLine($"  F((1,0) -> (0,0)) = {0.9 * Phi((0, 0)) - Phi((1, 0)):F3} (loin du but)");
Console.WriteLine($"  F((2,2) -> (3,2)) = {0.9 * Phi((3, 2)) - Phi((2, 2)):F3} (atteindre le but)");

Console.WriteLine();
Console.WriteLine("=> Le shaping recompense les mouvements vers le but,");
Console.WriteLine("   mais le theoreme garantit que la politique optimale est preservee.");

=== Reward Shaping avec Fonction de Potentiel ===

But en (3, 2)
Phi(s) = -distance(s, but)

Fonction de potentiel Phi(s) :
  -3,00   -2,00   -1,00   -0,00 
  -3,16   ####    -1,41   -1,00 
  -3,61   -2,83   -2,24   -2,00 

Exemples de shaping reward F(s, a, s') = gamma*Phi(s') - Phi(s) :
  F((0,0) -> (1,0)) = 1,060 (vers le but)
  F((1,0) -> (0,0)) = -0,417 (loin du but)
  F((2,2) -> (3,2)) = 1,000 (atteindre le but)

=> Le shaping recompense les mouvements vers le but,
   mais le theoreme garantit que la politique optimale est preservee.


## 8. Bandits Multi-Bras

### Le probleme

Vous avez K machines a sous ("bras"). Chaque bras i donne une recompense selon une distribution inconnue.

**Dilemme exploration/exploitation** :
- Explorer : essayer de nouveaux bras pour estimer leurs distributions
- Exploiter : jouer le meilleur bras connu

### Approches classiques

| Methode | Description |
|---------|-------------|
| ε-greedy | Exploiter avec proba 1-ε, explorer avec ε |
| UCB | Upper Confidence Bound : optimisme face a l'incertitude |
| Thompson | Echantillonner selon la posterior des recompenses |
| **Gittins** | Solution optimale pour bandits avec discount |

In [15]:
// Bandit multi-bras avec differentes strategies

public class MultiArmedBandit
{
    private double[] _trueMeans;
    private Random _rng;
    
    public int K { get; }
    
    public MultiArmedBandit(double[] means, int seed = 42)
    {
        _trueMeans = means;
        K = means.Length;
        _rng = new Random(seed);
    }
    
    public double Pull(int arm)
    {
        // Recompense gaussienne autour de la moyenne vraie
        double u1 = _rng.NextDouble();
        double u2 = _rng.NextDouble();
        double z = Math.Sqrt(-2 * Math.Log(u1)) * Math.Cos(2 * Math.PI * u2);
        return _trueMeans[arm] + 0.5 * z;
    }
    
    public double OptimalMean => _trueMeans.Max();
}

// Strategies
public interface IBanditStrategy
{
    int SelectArm(int[] counts, double[] sumRewards);
    string Name { get; }
}

public class EpsilonGreedy : IBanditStrategy
{
    private double _epsilon;
    private Random _rng = new Random();
    public string Name => $"ε-greedy (ε={_epsilon})";
    
    public EpsilonGreedy(double epsilon) { _epsilon = epsilon; }
    
    public int SelectArm(int[] counts, double[] sumRewards)
    {
        if (_rng.NextDouble() < _epsilon)
            return _rng.Next(counts.Length);
        
        int best = 0;
        double bestMean = double.NegativeInfinity;
        for (int i = 0; i < counts.Length; i++)
        {
            double mean = counts[i] > 0 ? sumRewards[i] / counts[i] : 0;
            if (mean > bestMean) { bestMean = mean; best = i; }
        }
        return best;
    }
}

public class UCB : IBanditStrategy
{
    public string Name => "UCB1";
    
    public int SelectArm(int[] counts, double[] sumRewards)
    {
        int totalPulls = counts.Sum();
        if (totalPulls < counts.Length)
            return totalPulls; // Essayer chaque bras une fois
        
        int best = 0;
        double bestUCB = double.NegativeInfinity;
        for (int i = 0; i < counts.Length; i++)
        {
            double mean = sumRewards[i] / counts[i];
            double bonus = Math.Sqrt(2 * Math.Log(totalPulls) / counts[i]);
            double ucb = mean + bonus;
            if (ucb > bestUCB) { bestUCB = ucb; best = i; }
        }
        return best;
    }
}

// Simulation
var bandit = new MultiArmedBandit(new[] { 0.3, 0.5, 0.7, 0.4 });
var strategies = new IBanditStrategy[] { new EpsilonGreedy(0.1), new UCB() };

Console.WriteLine($"Bandit avec {bandit.K} bras, moyennes vraies inconnues");
Console.WriteLine($"Meilleure moyenne : {bandit.OptimalMean}\n");

int T = 1000;

foreach (var strategy in strategies)
{
    var counts = new int[bandit.K];
    var sumRewards = new double[bandit.K];
    double totalReward = 0;
    double totalRegret = 0;
    
    for (int t = 0; t < T; t++)
    {
        int arm = strategy.SelectArm(counts, sumRewards);
        double reward = bandit.Pull(arm);
        
        counts[arm]++;
        sumRewards[arm] += reward;
        totalReward += reward;
        totalRegret += bandit.OptimalMean - reward;
    }
    
    Console.WriteLine($"{strategy.Name}:");
    Console.WriteLine($"  Recompense totale : {totalReward:F1}");
    Console.WriteLine($"  Regret cumule : {totalRegret:F1}");
    Console.WriteLine($"  Tirages par bras : {string.Join(", ", counts)}\n");
}

Bandit avec 4 bras, moyennes vraies inconnues
Meilleure moyenne : 0,7

ε-greedy (ε=0,1):
  Recompense totale : 684,6
  Regret cumule : 15,4
  Tirages par bras : 28, 69, 882, 21

UCB1:
  Recompense totale : 616,1
  Regret cumule : 83,9
  Tirages par bras : 54, 166, 719, 61



## 9. Indice de Gittins

### Le Theoreme Fondamental (Gittins, 1979)

L'indice de Gittins est l'un des resultats les plus elegants de la theorie de la decision. Il resout le probleme du bandit multi-bras de maniere **optimale** et **decomposable**.

> **Theoreme** : Pour un bandit multi-bras avec facteur de discount γ, la strategie optimale est de **toujours jouer le bras avec l'indice de Gittins le plus eleve**.

### Pourquoi est-ce remarquable ?

1. **Reduction de complexite** : Un probleme a K bras interdependants devient K problemes independants
2. **Optimalite garantie** : Pas une heuristique, c'est la solution exacte
3. **Index-ability** : Chaque bras a un "prix" intrinseque

### Definition intuitive

L'indice de Gittins d'un bras represente :

> "Le prix equivalent certain" de ce bras - la recompense garantie pour laquelle on serait indifferent entre jouer ce bras et recevoir cette recompense fixe.

Plus formellement : c'est le taux d'interet critique au-dessus duquel on prefererait "encaisser" plutot que jouer le bras.

### Calcul formel

L'indice est la solution d'un probleme d'arret optimal :

$$G(s) = \sup_{\tau \geq 1} \frac{E\left[\sum_{t=0}^{\tau-1} \gamma^t R_t \mid s_0 = s\right]}{E\left[\sum_{t=0}^{\tau-1} \gamma^t \mid s_0 = s\right]}$$

- Le numerateur est la recompense totale actualisee jusqu'au temps d'arret τ
- Le denominateur est le "temps effectif" actualise
- On maximise sur tous les temps d'arret possibles

### Implications pratiques

| Aspect | Consequence |
|--------|-------------|
| **Exploration** | Un bras incertain a un indice eleve (optimisme) |
| **Exploitation** | Un bras connu avec haute moyenne a un indice eleve |
| **Compromis** | L'indice equilibre automatiquement exploration/exploitation |

### Limitation

Le theoreme de Gittins s'applique sous des hypotheses specifiques :
- Discount γ < 1 (pas pour horizon fini sans discount)
- Les bras sont independants (pas d'interactions)
- Une seule action par etape (pas de contraintes de ressources)

## 10. POMDPs : MDPs Partiellement Observables

### Motivation

Dans un MDP standard, l'agent connait l'etat exact du monde a chaque instant. En pratique, c'est souvent **irrealiste** :

- Un robot ne voit pas a travers les murs
- Un medecin ne connait pas la vraie maladie, seulement les symptomes
- Un joueur de poker ne voit pas les cartes adverses

### Extension du MDP

Dans un POMDP, l'agent **ne connait pas l'etat exact**. Il recoit des **observations** qui sont des indicateurs bruitees de l'etat reel.

### Definition formelle

Un POMDP est un tuple (S, A, P, R, O, Ω, γ) :

| Element | Description |
|---------|-------------|
| S, A, P, R, γ | Comme MDP (etats, actions, transitions, recompenses, discount) |
| **O** | Ensemble des observations possibles |
| **Ω(o\|s', a)** | Modele de capteur : P(observation \| nouvel etat, action) |

### Belief State : La Cle des POMDPs

Puisque l'agent ne connait pas l'etat, il doit maintenir une **distribution de croyance** (belief) b(s) sur tous les etats possibles.

> b(s) = P(etat = s | historique des observations et actions)

Le belief state est une **statistique suffisante** : il resume toute l'information pertinente de l'historique.

### Mise a jour du Belief State

Apres avoir fait action a et observe o, le nouveau belief b' est :

$$b'(s') = \eta \cdot \Omega(o|s', a) \sum_s P(s'|s, a) b(s)$$

ou η est une constante de normalisation.

C'est exactement une **mise a jour bayesienne** ! Le POMDP est donc naturellement lie a l'inference probabiliste.

### Plans conditionnels

La politique d'un POMDP n'est pas s → a comme dans un MDP, mais b → a ou equivalemment un **arbre de decision** :

```
        a1           <- Action initiale
       /  \
     o1    o2        <- Observations possibles
     |      |
    a2     a3        <- Actions conditionnelles
   / \    / \
  o1 o2  o1 o2
  ...
```

### Complexite

Les POMDPs sont **PSPACE-complete** a resoudre exactement. En pratique, on utilise :
- Point-Based Value Iteration (PBVI)
- SARSOP
- Monte Carlo Tree Search (MCTS)
- Ou on approxime par des MDP avec belief states discrets

In [16]:
// POMDP simple : Tigre derriere une porte

Console.WriteLine("=== POMDP : Probleme du Tigre ===\n");
Console.WriteLine("Deux portes : gauche (G) et droite (D)");
Console.WriteLine("Un tigre est cache derriere l'une des portes.");
Console.WriteLine("Actions : Ouvrir G, Ouvrir D, Ecouter\n");

// Etats : tigre gauche (TG), tigre droite (TD)
// Actions : ouvrir_gauche, ouvrir_droite, ecouter
// Observations (si ecouter) : bruit_gauche, bruit_droite

double pCorrectHearing = 0.85; // P(bruit_gauche | tigre_gauche)
double rewardTresor = 10;
double rewardTigre = -100;
double costEcoute = -1;

// Belief state : b = P(tigre_gauche)
double b = 0.5; // Prior uniforme

Console.WriteLine($"Belief initial : P(tigre gauche) = {b:P0}\n");

// Fonction de valeur pour chaque action
double EU_ouvrirGauche(double belief) => 
    belief * rewardTigre + (1 - belief) * rewardTresor;

double EU_ouvrirDroite(double belief) => 
    belief * rewardTresor + (1 - belief) * rewardTigre;

// Pour ecouter, on doit calculer la valeur esperee apres observation
// (simplifie ici)

Console.WriteLine("Utilites esperees :");
Console.WriteLine($"  E[U(ouvrir gauche) | b={b:P0}] = {EU_ouvrirGauche(b):F1}");
Console.WriteLine($"  E[U(ouvrir droite) | b={b:P0}] = {EU_ouvrirDroite(b):F1}");
Console.WriteLine($"  Ecouter : cout immediat = {costEcoute}, mais reduit l'incertitude\n");

// Simuler une sequence d'observations
Console.WriteLine("Simulation : 3 ecoutes donnent 'bruit gauche'\n");

for (int i = 0; i < 3; i++)
{
    // Mise a jour bayesienne apres observation "bruit gauche"
    // P(TG|bruit_g) = P(bruit_g|TG) * P(TG) / P(bruit_g)
    double pBruitGauche = b * pCorrectHearing + (1 - b) * (1 - pCorrectHearing);
    b = (pCorrectHearing * b) / pBruitGauche;
    
    Console.WriteLine($"Apres observation {i+1} : P(tigre gauche) = {b:P1}");
}

Console.WriteLine();
Console.WriteLine($"E[U(ouvrir gauche)] = {EU_ouvrirGauche(b):F1}");
Console.WriteLine($"E[U(ouvrir droite)] = {EU_ouvrirDroite(b):F1}");
Console.WriteLine();
Console.WriteLine($"=> Decision : OUVRIR DROITE (tigre probablement a gauche)");

=== POMDP : Probleme du Tigre ===

Deux portes : gauche (G) et droite (D)
Un tigre est cache derriere l'une des portes.
Actions : Ouvrir G, Ouvrir D, Ecouter

Belief initial : P(tigre gauche) = 50 %

Utilites esperees :
  E[U(ouvrir gauche) | b=50 %] = -45,0
  E[U(ouvrir droite) | b=50 %] = -45,0
  Ecouter : cout immediat = -1, mais reduit l'incertitude

Simulation : 3 ecoutes donnent 'bruit gauche'

Apres observation 1 : P(tigre gauche) = 85,0 %
Apres observation 2 : P(tigre gauche) = 97,0 %
Apres observation 3 : P(tigre gauche) = 99,5 %

E[U(ouvrir gauche)] = -99,4
E[U(ouvrir droite)] = 9,4

=> Decision : OUVRIR DROITE (tigre probablement a gauche)


## 10bis. Belief State Updates avec Infer.NET

L'exemple precedent calculait les mises a jour du belief "a la main". Avec Infer.NET, on peut automatiser cette inference bayesienne, ce qui devient precieux pour des modeles plus complexes.

### Avantages d'Infer.NET pour les POMDPs

| Aspect | Calcul manuel | Infer.NET |
|--------|---------------|-----------|
| **Formule** | Ecrire Bayes explicitement | Automatique |
| **Etats multiples** | Combinatoire | Gere par le moteur |
| **Observations multiples** | Produits manuels | Modelisation naturelle |
| **Incertitude sur parametres** | Tres complexe | Hyper-priors possibles |

### Scenario : Maintenance predictive

Un systeme peut etre dans 3 etats : {Bon, Degrade, Defaillant}.
A chaque pas de temps :
- L'etat peut se degrader (Markov)
- On observe un signal de capteur (bruite)
- On decide : continuer, maintenance preventive, ou remplacement

In [ ]:
// Belief State Updates avec Infer.NET : Maintenance Predictive

using Microsoft.ML.Probabilistic.Math;

Console.WriteLine("=== Belief State Updates avec Infer.NET ===\n");
Console.WriteLine("Scenario : Maintenance predictive d'un systeme\n");

// Definition des etats et observations
int nEtats = 3;
Range etatRange = new Range(nEtats).Named("etatRange");
string[] etatsNom = { "Bon", "Degrade", "Defaillant" };

// Matrice de transition (degradation naturelle)
double[,] transMatrix = {
    // vers:    Bon    Degrade  Defaillant
    /* Bon */    { 0.90,  0.08,    0.02 },
    /* Deg */    { 0.00,  0.85,    0.15 },
    /* Def */    { 0.00,  0.00,    1.00 }  // Absorbant
};

// Modele d'observation : capteur de vibration (0=normal, 1=anormal)
double[,] obsMatrix = {
    // P(obs | etat)  Normal  Anormal
    /* Bon */       { 0.95,   0.05 },
    /* Deg */       { 0.30,   0.70 },
    /* Def */       { 0.05,   0.95 }
};

// Utilites des decisions
double[,] utilities = {
    //                  Bon     Degrade  Defaillant
    /* Continuer */   { 100,    50,      -500 },  // Risque si defaillant
    /* Maintenance */ { -20,    80,       -50 },  // Preventif, moins de gain si bon
    /* Remplacer */   { -100,  -100,       50 }   // Couteux mais resout defaillance
};
string[] actionsNom = { "Continuer", "Maintenance", "Remplacer" };

// === Simulation avec Infer.NET ===
// Observations sequentielles : Normal, Anormal, Anormal
int[] observations = { 0, 1, 1 };

// Belief initial (le systeme vient d'etre installe => Bon)
double[] belief = { 0.95, 0.04, 0.01 };

Console.WriteLine("Belief initial :");
for (int e = 0; e < nEtats; e++)
    Console.WriteLine($"  P({etatsNom[e]}) = {belief[e]:P1}");
Console.WriteLine();

// Boucle de mise a jour
for (int t = 0; t < observations.Length; t++)
{
    Console.WriteLine($"=== Pas de temps {t + 1} ===");
    
    // 1. Prediction (transition)
    double[] predicted = new double[nEtats];
    for (int sPrime = 0; sPrime < nEtats; sPrime++)
    {
        for (int s = 0; s < nEtats; s++)
            predicted[sPrime] += transMatrix[s, sPrime] * belief[s];
    }
    
    Console.WriteLine("Apres prediction (transition) :");
    for (int e = 0; e < nEtats; e++)
        Console.WriteLine($"  P({etatsNom[e]}) = {predicted[e]:P1}");
    
    // 2. Correction (observation) avec Infer.NET
    Variable<int> etatVar = Variable.Discrete(predicted).Named("etat");
    etatVar.SetValueRange(etatRange);
    
    Variable<int> obsVar = Variable.New<int>().Named("observation");
    obsVar.SetValueRange(new Range(2));
    
    // Modele d'observation
    using (Variable.Case(etatVar, 0))
        obsVar.SetTo(Variable.Discrete(obsMatrix[0, 0], obsMatrix[0, 1]));
    using (Variable.Case(etatVar, 1))
        obsVar.SetTo(Variable.Discrete(obsMatrix[1, 0], obsMatrix[1, 1]));
    using (Variable.Case(etatVar, 2))
        obsVar.SetTo(Variable.Discrete(obsMatrix[2, 0], obsMatrix[2, 1]));
    
    // Observation
    int obs = observations[t];
    obsVar.ObservedValue = obs;
    
    // Inference
    InferenceEngine engineBelief = new InferenceEngine();
    engineBelief.Compiler.CompilerChoice = Microsoft.ML.Probabilistic.Compiler.CompilerChoice.Roslyn;
    
    var posteriorEtat = engineBelief.Infer<Discrete>(etatVar);
    
    Console.WriteLine($"Observation : {(obs == 0 ? "Normal" : "Anormal")}");
    Console.WriteLine("Apres correction (Bayes via Infer.NET) :");
    for (int e = 0; e < nEtats; e++)
    {
        string bar = new string('#', (int)(posteriorEtat.GetProbs()[e] * 30));
        Console.WriteLine($"  P({etatsNom[e],-10}) = {posteriorEtat.GetProbs()[e]:P1} {bar}");
    }
    
    // 3. Decision basee sur le belief
    double[] EU = new double[3];
    for (int a = 0; a < 3; a++)
    {
        for (int e = 0; e < nEtats; e++)
            EU[a] += posteriorEtat.GetProbs()[e] * utilities[a, e];
    }
    
    int bestAction = EU.Select((v, i) => (v, i)).OrderByDescending(x => x.v).First().i;
    
    Console.WriteLine("Utilites esperees :");
    for (int a = 0; a < 3; a++)
        Console.WriteLine($"  E[U({actionsNom[a],-12})] = {EU[a],7:F1}");
    Console.WriteLine($"=> Decision : {actionsNom[bestAction]}");
    Console.WriteLine();
    
    // Mettre a jour le belief pour le prochain pas
    belief = posteriorEtat.GetProbs();
}

Console.WriteLine("=== Resume ===");
Console.WriteLine("Les observations 'Anormal' successives ont fait augmenter P(Degrade),");
Console.WriteLine("declenchant le passage de 'Continuer' a 'Maintenance' ou 'Remplacer'.");
Console.WriteLine("\nC'est le principe de la maintenance predictive bayesienne !");

## 11. Lien avec la Serie RL

### Ce notebook : Concepts fondamentaux

- MDPs et equations de Bellman
- Methodes tabulaires (Value Iteration, Policy Iteration)
- Concepts theoriques (Gittins, POMDPs)

### Serie RL (`MyIA.AI.Notebooks/RL/`) : Implementations avancees

| Notebook | Contenu |
|----------|--------|
| RL-1-Introduction | Q-Learning tabulaire |
| RL-2-DeepRL | DQN avec reseaux de neurones |
| RL-3-PolicyGradient | REINFORCE, Actor-Critic |
| RL-4-AdvancedMethods | PPO, A2C, SAC |
| RL-5-Applications | Gym, Stable-Baselines3 |

## 12. Resume

| Concept | Description |
|---------|-------------|
| **MDP** | (S, A, P, R, γ) - cadre formel pour decisions sequentielles |
| **Bellman** | V*(s) = max_a [R + γ Σ P V*] |
| **Value Iteration** | Mise a jour iterative de V jusqu'a convergence |
| **Policy Iteration** | Evaluation + Amelioration alternees |
| **Reward Shaping** | F = γΦ(s') - Φ(s) preserve la politique optimale |
| **Bandits** | Exploration vs Exploitation |
| **Gittins** | Indice optimal pour bandits avec discount |
| **POMDP** | MDP avec observations bruitees, belief states |

---

## Pour aller plus loin

| Si vous voulez... | Consultez... |
|-------------------|-------------|
| Deep Reinforcement Learning | Serie `MyIA.AI.Notebooks/RL/` |
| Implementations PyTorch/TF | Stable-Baselines3 |
| Theorie avancee | Sutton & Barto "Reinforcement Learning" |

---

## Fin de la Serie Decision Theory

Felicitations ! Vous avez termine les 7 notebooks sur la Decision Theory.

### Recapitulatif de la serie 14-20

| # | Titre | Concepts cles |
|---|-------|---------------|
| 14 | Utility Foundations | Axiomes VNM, loteries, agent rationnel |
| 15 | Utility Money | CARA/CRRA, aversion au risque, dominance |
| 16 | Multi-Attribute | MAUT, independance, SMART |
| 17 | Decision Networks | Influence diagrams, arcs informationnels |
| 18 | Value of Information | EVPI, EVSI, droits de forage |
| 19 | Expert Systems | Minimax, regret, robustesse |
| 20 | Sequential | MDPs, bandits, POMDPs |

---

## References

- Bellman (1957) : Dynamic Programming
- Gittins (1979) : Bandit Processes and Dynamic Allocation Indices
- Ng, Harada, Russell (1999) : Policy Invariance Under Reward Transformations
- Kaelbling, Littman, Cassandra (1998) : Planning and Acting in Partially Observable Stochastic Domains
- Sutton & Barto (2018) : Reinforcement Learning: An Introduction